# Tools
## Regular Expressions
find a string -> `s.find('word')`

### difference between seacrh & match in re

`search`: return True if pattern appears anywhere 

`match`: return True if pattern appears at the start of text.


In [2]:
import re
help(re.search)

Help on function search in module re:

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a Match object, or None if no match was found.



In [3]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a Match object, or None if no match was found.



In [29]:
from re import search, match
test_expr = r"baa*!"
def test():
    assert search(test_expr, "sheep said baaaa!")
    assert bool(search(test_expr,"sheep said baaaa humbug")) == False
    assert bool(match(test_expr,"sheep said baaaa!")) == False
    assert match(test_expr,"baaaaa! said the sheep")
    print("test passed")
test()

test passed


In [24]:
assert bool(search(test_expr,"sheep said baaaa humbug"))==False

In [18]:
search(test_expr,"sheep said baaaa humbug") 

In [31]:
def search(pattern, text):
    if pattern.startswith("^"):
        return match(pattern[1:],text)
    else:
        return match(".*"+pattern,text) # 如果text start with \n 呢。 .*并不能匹配\n
search(test_expr, "\nbaaaaa! said the sheep")

In [ ]:
def match(pattern, text):
    if pattern == "":
        return True
    elif pattern == "$":
        return (text=="")
    elif len(pattern) > 1 and pattern[1] in "*?":
        p, op, pat = patter[0], pattern[1], pattern[2:]
        if op=="*":
            return match_star(p, pat, text)
        elif op=="?":
            if match1(p, text) and match(pat, text[1:]):
                return True
            else:
                return match(pat,text)
    else:
        (match1(pattern[0], text) and 
        match(pattern[1:], text[1:]))
def match1(p, text):
    if not text: return False
    return p== "." or p == text[0]

def match_star(p, pattern, text):
    return (match(pattern, text) or (match1(p, text) and match_start(p, pat, test[1:]))

## language
### python

statement

expressions

format 

class- operator overloading



### inventory of concepts
pattern : text -> result 

partial result -> matchset(pattern, text) return set of remainders

control over iteration

### RE API
|func | example | set |
|----|------|-------|
|lit(s)| lit("a")|{a}|
|seq(x,y)| seq(lit("a"),lit("b"))| {ab}|
|alt(x,y) | alt(-, -)| {a, b}|
|star(x) | star(lit("a"))| {a, aa,aaa}|
| oneof(c)| oneof("abc")| {a, b, c}|
| eol | seq(lit("a"),eol)| {a}|

In [6]:
null = frozenset()
def components(pattern):
    "patterns are passed in nested list or tuple"
    x = pattern[1] if len(pattern) > 1 else None
    y = pattern[2] if len(pattern) > 2 else None
    return pattern[0], x, y
def matchset(pattern, text):
    op, x, y = components(pattern)
    if "lit" == op:
        return set([text[len(x):]]) if text.startswith(x) else null
    elif "seq" == op:
        return set(t2 for t1 in matchset(x, text) for  t2 in matchset(y,t1))
    elif "alt" == op:
        return matchset(x, text) | matchset(y, text)
    elif "dot" == op:
        return set([text[1:]]) if text else null
    elif "oneof" == op:
        return set([""]) if text.start  else null
    elif "eol" == op:
        return set([""]) if text =="" else null 

    elif "star" == op:
        return (set([text]) | set(t2 for t1 in matchset(x, text) 
                                 for t2 in matchset(pattern,t1))) 
    # star 可能会重复多次，但没有关系，继续递归匹配子串
    else:
        raise ValueError("unknown pattern: %s" % pattern)

In [10]:
def lit(string):  return ('lit', string)
def seq(x, y):    return ('seq', x, y)
def alt(x, y):    return ("alt", x, y)
def star(x):      return ("star", x)
def plus(x):      return ("seq", x, ("star", x))
def opt(x):       return alt(lit(''), x) #opt(x) means that x is optional
def oneof(chars): return ('oneof', tuple(chars))
dot = ('dot',)
eol = ("eol",)

def test():
    assert lit('abc')         == ('lit', 'abc')
    assert seq(('lit', 'a'), 
               ('lit', 'b'))  == ('seq', ('lit', 'a'), ('lit', 'b'))
    assert alt(('lit', 'a'), 
               ('lit', 'b'))  == ('alt', ('lit', 'a'), ('lit', 'b'))
    assert star(('lit', 'a')) == ('star', ('lit', 'a'))
    assert plus(('lit', 'c')) == ('seq', ('lit', 'c'), 
                                  ('star', ('lit', 'c')))
    assert opt(('lit', 'x'))  == ('alt', ('lit', ''), ('lit', 'x'))
    assert oneof('abc')       == ('oneof', ('a', 'b', 'c'))
    return 'tests pass'

In [11]:
test()

'tests pass'

### review on api design
<img src="https://raw.githubusercontent.com/NickChilling/notes/master/pic_bed/20190920135317.png"/>

In [ ]:

def search(pattern, text):
    "Match pattern anywhere in text; return longest earliest match or None."
    for i in range(len(text)):
        m = match(pattern, text[i:])
        if m is not None: # include ""
            return m
        
def match(pattern, text):
    "Match pattern against start of text; return longest match found or None."
    remainders = matchset(pattern, text)
    if remainders:
        shortest = min(remainders, key=len)
        return text[:len(text)-len(shortest)]
    
def test():
    assert match(('star', ('lit', 'a')),'aaabcd') == 'aaa'
    assert match(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == None
    assert match(('alt', ('lit', 'b'), ('lit', 'a')), 'ab') == 'a'
    assert search(('alt', ('lit', 'b'), ('lit', 'c')), 'ab') == 'b'
    return 'tests pass'

`matchset`中存在一个问题是：每次调用都需要重新解析`pattern`。

当我们需要对不同的文本解析相同的模式时，这样做时非常低效的。(解释)

因此，可以考虑先将`pattern`解析，每次调用`matchset`传入的是解析的结果。 (编译)


In [23]:
def lit(s): return lambda text: set([text[len(s):]]) if text.startswith(s) else null
# 返回一个函数，而不是string
def seq(x, y): return lambda text: set().union(*map(y, x(text)))    # why use set().union 

def alt(x, y): return lambda text: set().union(x(text),y(text))

def oneof(chars): return lambda t: set([t[1:]]) if (t and t[0] in chars)

dot = lambda t: set([t[1:]]) if t else null

eol = lambda t: set([""]) if t=="" else null

def star(x): return lambda t: (set[t]) | set(t2 for t1 in x[t] if t1!=t for t2 in star(x)(t1))
def test():
    g = alt(lit('a'), lit('b'))
    print(g("abc"))
    assert g('abc') == set(['bc'])
    return 'test passes'
print(test())

{'bc'}
test passes


上面这个思路实际上来说，就是将原本统一由`matchset`通过`if else`进行的`tuple`-> `string`的转换改成了通过`return a funciton`在每个api中进行转换。

之前实现的是`recoginzer` ， 现在开始尝试实现`generator`
对于`a*`这样拥有无限个结果的`pattern` .我们可以考虑用`Python`中的生成器来实现。但实际上，可以给生成器加上一个定长参数

In [ ]:
def lit(s): return lambda Ns:set([s]) if len(s) in Ns else null
def alt(x, y): return lambda Ns: x(Ns) | y(Ns)
def star(x): return lambda Ns: opt(plus(x))(Ns)
def plus(x): return lambda Ns: genseq(x, star(x), Ns, startx=1)
def oneof(chars): return lambda Ns: set(chars) if 1 in Ns else null
def opt(x): return alt(epsilon,x)
dot = oneof("?")
epsilon = lit("")

`lit`中的`set()`与`text`是无关的。因此应该放到编译过程中进行

In [24]:
def genseq(x, y, Ns):
    Nss = range(max(Ns)+1)
    return set(m1+m2 for m1 in x(Nss) for m2 in y(Nss) if len(m1+m2) in Ns)


上面这个函数在有结果的情况下，是正确的。但可能会陷入递归出不来
对于像 `a = lit("a), pat = plus(opt(a))`
`generate  a = ""` 则退不出递归

In [ ]:
def genseq(x, y, Ns, startx=0 ):
    if not Ns:
        return Null
    xmatches = x(set(range(startx, max(Ns)+1)))
    Ns_x = set(len(m) for m in xmatches)
    Ns_y = set(n-m for n in Ns for m in Ns_x if n-m>=0)
    ymatches = y(Ns_y)
    return set(m1+m2 for m1 in xmatches for m2 in ymatches
              if len(m1+m2) in Ns)


seq is binary 

now: if we want to call seq -> `seq(a, seq(b, seq(c,d)))`

what we want : `seq(a,b,c,d)`

two things we need to consider changes on `seq` 

1. backward compatible : for those who call `seq` like `a`->`seq` , there is no difference after we changed `seq` 
2. internal/external(changes to interface)

对于`matchset`这个版本的`seq`而言，我们可以做一个映射函数，做一个多元的`seq`的根据传参数量来返回二元的`seq`，这样就做到了`backward compatible` 但问题在于，在我们的API中，还有许多函数是二元的。难道我们需要对这些函数一个个修改吗？

DRY!!!(don't repeat your self)

写一个统一将binary函数 改为nary函数的 wrapper


In [34]:
def n_ary(f):
    def n_ary_f(x, *args):
        return x if not args else f(x, n_ary_f(*args)) # *args 在于自动解包
    return n_ary_f

In [35]:
@n_ary
def seq_1(x, y): return ('seq', x, y)


In [36]:
# problem is 
help(seq_1)

Help on function n_ary_f in module __main__:

n_ary_f(x, *args)



In [38]:
def n_ary_1(f):
    def n_ary_f(x, *args):
        return x if not args else f(x, n_ary_f(*args)) # *args 在于自动解包
    from functools import update_wrapper
    update_wrapper(n_ary_f, f) # copy documetation
    return n_ary_f

In [39]:
@n_ary_1
def seq_1(x, y): return ('seq', x, y)

In [40]:
help(seq_1)

Help on function seq_1 in module __main__:

seq_1(x, y)



对每个装饰器加上一个`update_wrapper`仍然略显重复。那么，我们可以`@decorator`直接声明这是一个装饰器

In [43]:
from functools import update_wrapper
def decorator(d):
    def _d(fn):
        return update_wrapper(d(fn),fn)
    update_wrapper(_d, d)
    return _d
@decorator
def n_ary_2(f):
    def n_ary_f(x, *args):
        return x if not args else f(x, n_ary_f(*args)) 
    return n_ary_f
help(n_ary_2)

Help on function n_ary_2 in module __main__:

n_ary_2(f)



所谓`decorator`实际上就是返回函数的函数。第一次调用时，只返回一个函数，第二次调用才产生结果。
而`decorator`这是将这两个步骤统一起来。两
个函数的参数同时传递。

In [48]:
@decorator
def memo(f):
    memo_dict = {}
    def wrapper(*args):
        if args[0] in memo_dict:
            return memo_dict[args[0]]
        else:
            result = f(*args)
            memo_dict[args[0]] = result
            return result
    return wrapper

@memo
def fib(n): return fib(n-1)+fib(n-2) if n>2 else 1

In [51]:
fib(10)

55

In [53]:
def trace(f):
    indent = "\t"
    def _f(*args):
        signature = "%s(%s)" % (f.__name__, ", ".join(map(repr,args)))
        print("%s--> %s" %(trace.level*indent, signature))
        trace.level += 1
        try:
            result = f(*args)
            print("%s<-- %s === %s " % ((trace.level-1)*indent, signature, result))
        finally:
            trace.level -= 1
        return result
    trace.level = 0
    return _f

### unhashable 
list is unhashable because it's mutable
### types of tool

debug -> countcalls, trace ,disabled

performance -> memo

### disable decorator


In [ ]:
@decorator
def disable(f):
    return f
trace = disable # disable trace 